# Creating a list of unique users with their activity statistics (with a cut-off date)

This notebook scans through all collected user activity files and produces a single csv with user-level statistics:
 - number of posts
 - number of comments
 - total post karma
 - total comment karma
 - first and last date

+ MODIFICATION - only consider posts before 2022 January 10 (timestamp < 1641790800)

In [2]:
import polars as pl
from pathlib import Path
import datetime
import time
import sqlite3
from tqdm import tqdm
import pandas as pd

import datetime
import time
import re

path = "../../../data/users/"
user_stats = path + 'users.sqlite.db'

In [ ]:
files = [f.absolute() for f in Path(path).joinpath("raw/").glob("*.csv")]

max_batch_id = 9979

r = re.compile("user_interactions-batch-20-(\d+).csv")
files = [f for f in files if int(r.findall(str(f))[0]) <= max_batch_id]
print("Total files to process: {}".format(len(files)))

In [ ]:
cols_to_read = ["user_name", "no_posts", "no_comments", "post_karma", "comment_karma", "first_date", "last_date"] #skip the large json columns

def get_file(f):
    return pl.scan_csv(f).select(cols_to_read).with_columns([
        pl.col("first_date").apply(lambda x: int(time.mktime(datetime.date.fromisoformat(x).timetuple()))),
        pl.col("last_date").apply(lambda x: int(time.mktime(datetime.date.fromisoformat(x).timetuple())))
    ]).collect()

#read in the first file to serve as the baseline
df = get_file(files[0])

#read in all other files and groupby / add up statistics after each file
for f in tqdm(files[1:]):    
    new_df = get_file(f)
    df.vstack(new_df, in_place=True)
    df = df.lazy().groupby("user_name").agg([
        pl.col("no_posts").sum(),
        pl.col("no_comments").sum(),
        pl.col("post_karma").sum(),
        pl.col("comment_karma").sum(),
        pl.col("first_date").min(),
        pl.col("last_date").max()
    ]).collect() 

In [ ]:
df = df.with_columns([(pl.col("no_posts") + pl.col("no_comments")).alias("total_activity")])

In [ ]:
conn = sqlite3.connect(user_stats)
df.to_pandas().to_sql("users_cutoff", conn, index=False)

In [3]:
with sqlite3.connect(user_stats) as conn:
    cur = conn.cursor()        
    try:
        cur.execute("ALTER TABLE users_cutoff ADD COLUMN avg_post_karma real")
        cur.execute("ALTER TABLE users_cutoff ADD COLUMN avg_comment_karma real")
        cur.execute("ALTER TABLE users_cutoff ADD COLUMN activity_window real")                
        cur.execute("ALTER TABLE users_cutoff ADD COLUMN longevity real")        
    except sqlite3.OperationalError:
        print("columns already exist")
    
    cur.execute("CREATE INDEX IF NOT EXISTS user_name_index ON users_cutoff(user_name)")
    
    cur.execute("UPDATE users_cutoff SET avg_post_karma = CASE WHEN no_posts > 0 THEN post_karma / CAST(no_posts as REAL) ELSE 0 END")
    cur.execute("UPDATE users_cutoff SET avg_comment_karma = CASE WHEN no_comments > 0 THEN comment_karma / CAST(no_comments as REAL) ELSE 0 END")
    cur.execute("UPDATE users_cutoff SET activity_window = CAST(last_date - first_date as REAL) / CAST(3600 * 24 as REAL)") 
    cur.execute("UPDATE users_cutoff SET longevity = (1641790800.0 - first_date) / CAST(3600 * 24 as REAL)")